**<font color = black size=6>实验六:随机森林</font>**

In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random

**<font color = blue size=4>第一部分:函数介绍</font>**

介绍一些可能会用到的函数。

In [57]:
# np.random.choice函数从一个一维数组中随机采样
x = np.array([1,2,3,4])
y = np.random.choice(x, replace=True, size=10)
print(y)

# np.random.shuffle函数对一个数组/矩阵按照第一维进行洗牌
x = np.array([[0,1,2],[3,4,5],[6,7,8],[9,10,11],[12,13,14]])
np.random.shuffle(x)
print(x)

# DataFrame对象的sample函数可以随机采样n个数据或者采样比例为frac的数据
x = np.array([[0,0,0],[1,1,1],[2,2,2],[3,3,3],[4,4,4]])
frame = pd.DataFrame(x)
print(frame.sample(n=2))
print(frame.sample(frac=0.3))

[1 3 4 4 1 2 2 4 2 2]
[[ 9 10 11]
 [ 0  1  2]
 [12 13 14]
 [ 6  7  8]
 [ 3  4  5]]
   0  1  2
2  2  2  2
4  4  4  4
   0  1  2
1  1  1  1
2  2  2  2


**<font color = blue size=4>第二部分:实验任务</font>**

本次实验承接上次实验，实现随机森林。

<span style="color:purple">本次实验依旧使用泰坦尼克号数据集(train_titanic.csv, test_titanic.csv。数据集包括了四个属性特征以及一个标签(即为Survived,代表是否生还),属性特征分别为Sex(性别)，sibsp(堂兄弟妹个数)，Parch(父母与小孩的个数)，Pclass(乘客等级)  
其中该数据集无缺失值和异常值，且所有连续变量已自动转换为离散变量,标签(Survived)也自动转变为离散变量0和1，所以你无需进行数据预处理，可以直接使用该数据集。</span>

<span style="color:purple">1) 对上次实验的best_split函数进行修改，实现随机特征选择。  
先从特征集$A$中先随机选取$k$个特征构成特征集$A'$，再从$A'$中选取最佳划分的特征。$k$一般取$max\{log_2 d,1\}$, $d$是$A$的元素的个数。你可使用特征的信息增益来决定最佳划分的特征。  
    【输入】：数据集D、特征集A    
    【输出】：随机特征集A'中最佳划分的特征维数   
    【信息增益公式】:  
        某数据集D有若干特征值以及对应的标签值，其总样本大小为|D|,这里取其中一个特征feature,该特征包含V个不同的取值，特征值为第v(v=1,2,...,V)个值的数量为|$D^v$|$(\sum_{v=1}^VD^v=|D|)$,则该特征对应的信息增益为$$Gain(D,feature)=Ent(D)-\sum_{v=1}^K \frac{|D^v|}{D} Ent(D^v)$$  
    【信息熵公式】:  
        某数组包含K个不同的取值，样本为第k(k=1,2,...,K)个值的数量所占比例为p_k,则其信息熵为$$Ent=-\sum_{k=1}^K p_k log_2 p_k$$
</span>

In [58]:
# 计算标签数组的信息熵
def entropy(label):
    unique_classes, counts = np.unique(label, return_counts=True)
    total_samples = len(label)
    entropy_value = 0
    
    for count in counts:
        p_k = count / total_samples
        entropy_value -= p_k * np.log2(p_k)

    return entropy_value

# 计算考虑属性信息后的条件熵、给定维度的条件下的信息熵
def conditional_entropy(x_data, y_label, dimension):
    
    # 获取某个特征的所有可能取值
    unique_values = np.unique(x_data[:, dimension])
    total_samples = len(y_label)
    cond_entropy_value = 0

    for value in unique_values:
        # 获取相应特征的标签值
        sub_indices = np.where(x_data[:, dimension] == value)[0]
        sub_labels = y_label[sub_indices]
        
        # 计算占比
        sub_samples = len(sub_labels)
        p = sub_samples / total_samples
        
        # 计算条件熵
        sub_entropy = entropy(sub_labels)
        cond_entropy_value += p * sub_entropy

    return cond_entropy_value

# 选择信息增益最大的特征输出
def one_split_ID3(x_data, y_label):
    num_features = x_data.shape[1]
    total_entropy = entropy(y_label)
    best_info_gain = 0
    best_dimension = None

    for dimension in range(num_features):
        cond_entropy = conditional_entropy(x_data, y_label, dimension)
        info_gain = total_entropy - cond_entropy

        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_dimension = dimension
        
    return best_info_gain, best_dimension


def best_split(D, A):
    
    best_info_gain = 0
    best_dimension = None

    k = max(int(np.log2(len(A))), 1)
    A_new = random.sample(list(A), k)

    for dimension in A_new:
        x_data = D[:, dimension].reshape(-1, 1)
        info_gain, _ = one_split_ID3(x_data, D[:, -1])
        
        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_dimension = dimension
    
    return best_dimension

<span style="color:purple">2) 对上次实验完成的决策树类进行修改。你需要实现下面三个函数：  
1. TreeGenerate(self, D, A)：递归构建决策树，伪代码参照提供的“Algorithm 1 决策树学习基本算法”。  
2. train(self, D)：做一些数据预处理，包括将Dataframe转换为numpy矩阵，从数据集中提取属性集，并调用TreeGenerate函数来递归地生成决策树。  
3. predict(self, D)：对测试集D进行预测，要求返回数据集D的预测标签，即一个(|D|,1)矩阵（|D|行1列）。  
由于训练集是采样生成，因此需要对predict函数做修改。需要考虑测试集中出现决策树无法划分的特征值时的情况。给出两种参考的做法：  
a).对其不再进行预测，直接给定划分失败的样本标签(例如-1)。  
b).跳过该划分节点，随机选取一个特征值继续遍历。</span>

In [59]:
# 记下所有属性可能的取值
train_frame = pd.read_csv('train_titanic.csv')
D = np.array(train_frame)
A = set(range(D.shape[1] - 1))
possible_value = {}
for every in A:
    possible_value[every] = np.unique(D[:, every])

In [60]:
# 树结点类
class Node:
    def __init__(self, isLeaf=True, label=-1, index=-1):
        self.isLeaf = isLeaf # isLeaf表示该结点是否是叶结点
        self.label = label # label表示该叶结点的label（当结点为叶结点时有用）
        self.index = index # index表示该分支结点的划分属性的序号（当结点为分支结点时有用）
        self.children = {} # children表示该结点的所有孩子结点，dict类型，方便进行决策树的搜索
        
    def addNode(self, val, node):
        self.children[val] = node #为当前结点增加一个划分属性的值为val的孩子结点

In [61]:
# 决策树类
class DTree:
    def __init__(self):
        self.tree_root = None #决策树的根结点
        self.possible_value = possible_value # 用于存储每个属性可能的取值
    
        
    '''
    TreeGenerate函数用于递归构建决策树,伪代码参照课件中的“Algorithm 1 决策树学习基本算法”
    '''
    def TreeGenerate(self, D, A):
        
        node = Node(isLeaf=False)

        # if D中样本全属于同一类别C then
        if len(np.unique(D[:, -1])) == 1:
            node.isLeaf = True
            node.label = D[0, -1]  # 叶结点的类别为D中唯一的类别,取第一行最后一列
            return node
        # end if

        # if A = Ø OR D中样本在A上取值相同 then
        if len(A) == 0 or len(np.unique(D[:, list(A)])) == 1:
            node.isLeaf = True
            node.label = np.argmax(np.bincount(D[:, -1]))  # 叶结点的类别为D中样本最多的类
            return node
        # end if

        # 从A中选择最优划分特征a_star
        a_star = best_split(D, A)
        if a_star is None:
            node.isLeaf = True
            node.label = np.argmax(np.bincount(D[:, -1]))  # 叶结点的类别为D中样本最多的类
            return node

        node.index = a_star  # 设置分支节点的划分特征

        # for a_star 的每一个值a_star_v do
        for a_star_v in self.possible_value[a_star]:
            # D_v是D在a_star上取值为a_star_v的样本子集
            D_v = D[D[:, a_star] == a_star_v]
            
            if len(D_v) == 0:
                child_node = Node(isLeaf=True, label=np.argmax(np.bincount(D[:, -1])))
            else:
                A_new = A - {a_star}
                child_node = self.TreeGenerate(D_v, A_new)

            node.addNode(a_star_v, child_node)
        # end for
        
        return node
    
    
    '''
    train函数可以做一些数据预处理(比如Dataframe到numpy矩阵的转换,提取属性集等),并调用TreeGenerate函数来递归地生成决策树
    '''
    def train(self, D):
        # 将Dataframe对象转换为numpy矩阵（也可以不转，自行决定做法）
        D = np.array(D) 
        
        # 特征集A
        A = set(range(D.shape[1] - 1))
        
        #记下每个特征可能的取值
        for every in A:
            self.possible_value[every] = np.unique(D[:, every])

        # 递归地生成决策树，并将决策树的根结点赋值给self.tree_root
        self.tree_root = self.TreeGenerate(D, A)
        
        return
    
    
    '''
    predict函数对测试集D进行预测,输出预测标签
    '''
    def predict(self, D):
        # 将测试集 D 转化为 NumPy 数组
        D = np.array(D)
        # 用于存储预测的标签
        predictions = []

        # 对测试集中的每个样本进行预测
        for i in range(len(D)):
            # 从根节点开始遍历决策树
            current_node = self.tree_root

            # 循环遍历树，直到达到叶节点为止
            while not current_node.isLeaf:
                # 获取当前节点的划分特征的索引
                feature_index = current_node.index
                # 获取当前样本在划分特征上的取值
                feature_value = D[i, feature_index]

                if feature_value in current_node.children:
                    # 如果样本的特征值在当前节点的孩子节点中，继续向下遍历
                    current_node = current_node.children[feature_value]
                else:
                    # 如果样本的特征值不在当前节点的孩子节点中,采取随机选择一个特征值继续遍历的策略
                    available_features = list(current_node.children.keys())
                    random_feature_value = np.random.choice(available_features)
                    current_node = current_node.children[random_feature_value]

            # 到达叶节点后，预测标签即为叶节点的类别
            predicted_label = current_node.label
            # 将预测标签添加到预测结果中
            predictions.append(predicted_label)

        # 返回预测结果，将其转化为 NumPy 数组并重塑形状
        return np.array(predictions).reshape(-1, 1)

        

<span style="color:purple">3) 重复采用Bootstrap自助采样法对训练数据集'train_titanic.csv'进行采样，生成$n$个子训练数据集($n$自行设定)。  
Bootstrap采样法是指，每次从原数据集中【有放回】地随机采样一个样本，重复进行$m$次，就生成一个有$m$个样本的子数据集。</span>

In [62]:

train_frame = pd.read_csv('train_titanic.csv')

# 生成n个子训练数据集
n = 5

# 存储生成的子训练数据集的列表
sub_train_data_sets = []

# 使用自助采样法生成子训练数据集
for i in range(n):
    # 随机有放回地抽取与原始数据集相同大小的样本
    bootstrap_sample = train_frame.sample(frac=1, replace=True)
    sub_train_data_sets.append(bootstrap_sample)


<span style="color:purple">4) 生成n棵决策树实例，使用上述生成的n个子训练数据集各自训练一棵决策树，即子训练集D1训练决策树1，子训练集D2训练决策树2……</span>

In [63]:
# ----- Your code here -------

# 创建n棵决策树的列表
decision_trees = []

# 循环训练n棵决策树
for i in range(n):
    # 创建一个新的决策树实例
    decision_tree = DTree()
    
    # 获取第i个子训练数据集
    sub_train_data = sub_train_data_sets[i]
    
    # 训练决策树
    decision_tree.train(sub_train_data)
    
    # 将训练好的决策树添加到列表中
    decision_trees.append(decision_tree)


<span style="color:purple">5) 用训练完成的$n$棵决策树分别对测试数据集'test_titanic.csv'进行预测。采用相对多数投票法来对各棵决策树的预测结果进行结合。输出结合的预测结果的准确率。  
【相对多数投票法】  
对于某个样本$x$, 相对多数投票法预测它的标签为得票最多的标签。若同时有多个标签获得最高票，则从中随机选取一个。其公式如下所示：
$$H(x)=C_{\mathop{\arg\max}_{j} \sum_{i=1}^n h_i^j(x)}$$  
</span>

In [64]:
# 读取测试数据集
test_frame = pd.read_csv('test_titanic.csv')

# 用训练好的决策树进行预测
predictions = []

for i in range(len(test_frame)):
    # 统计每个决策树的预测结果
    votes = {}
    for j in range(n):
        decision_tree = decision_trees[j]
        prediction = decision_tree.predict(test_frame.iloc[i:i+1, :])
        label = prediction[0, 0]
        if label in votes:
            votes[label] += 1
        else:
            votes[label] = 1
    
    # 找到得票最多的标签
    max_votes = max(votes.values())
    winning_labels = [label for label, vote_count in votes.items() if vote_count == max_votes]
    
    # 从得票最多的标签中随机选取一个作为预测结果
    final_prediction = random.choice(winning_labels)
    
    predictions.append(final_prediction)

# 计算准确率
correct_labels = test_frame['Survived'].values
predicted_labels = np.array(predictions)

accuracy = np.mean(correct_labels == predicted_labels)
print("准确率：", accuracy)

准确率： 0.8415841584158416


In [ ]:
# 使用python内置库进行验证，结果基本一致
from sklearn.ensemble import  RandomForestClassifier  # 随机森林分类器
from sklearn.metrics import classification_report  # 生产报告

# 假设最后一列是标签列，将其分离出来
y_train = train_frame.iloc[:, -1]
y_test = test_frame.iloc[:, -1]

# 从DataFrame中去除最后一列以获得属性
x_train = train_frame.iloc[:, :-1]
x_test = test_frame.iloc[:, :-1]

# 3.训练分类器
rfc = RandomForestClassifier(n_jobs=-1)
train_history = rfc.fit(x_train, y_train)

# 4.测试
pred = rfc.predict(x_test)
report = classification_report(y_test, pred)

print(report)

"""
              precision    recall  f1-score   support

           0       0.85      0.96      0.90       158
           1       0.72      0.41      0.52        44

    accuracy                           0.84       202
   macro avg       0.79      0.68      0.71       202
weighted avg       0.82      0.84      0.82       202
"""

              precision    recall  f1-score   support

           0       0.85      0.96      0.90       158
           1       0.72      0.41      0.52        44

    accuracy                           0.84       202
   macro avg       0.79      0.68      0.71       202
weighted avg       0.82      0.84      0.82       202



**<font color = blue size=4>第三部分:作业提交</font>**

一、实验课下课前提交完成代码，如果下课前未完成，请将已经完成的部分进行提交，未完成的部分于之后的实验报告中进行补充  
要求:  
1)文件格式为：学号-姓名.ipynb  
2)不要提交文件夹、压缩包、数据集等无关文件，只需提交单个ipynb文件即可

二、实验报告提交截止日期为：【10月27日 14:20】  
提交地址：https://send2me.cn/g_kfMtFI/SuiqyPO6B7rxqg  
要求：  
1)文件格式为：学号-姓名-实验六.pdf  
2)【不要】提交文件夹、压缩包、代码文件、数据集等任何与实验报告无关的文件，只需要提交单个pdf文件即可  

三、课堂课件获取地址: https://www.jianguoyun.com/p/DTGgCYAQp5WhChir26EFIAA  
实验内容获取地址: https://www.jianguoyun.com/p/DekWAFoQp5WhChis26EFIAA